In [1]:
def value(s,sp, Ra, V, gamma, P ): 
    
    if(sp[0] < 0 or sp[0] >= len(Ra[0])): # FLYTTA ?
        return -1
       
    if(sp[1] < 0 or sp[1] >= len(Ra)):
        return -1
    
    return P*(Ra[sp[0]][sp[1]]+ gamma * V[sp[0]][sp[1]]) + (1-P)* (Ra[s[0]][s[1]] + gamma * V[s[0]][s[1]])

In [2]:
#Note. You need to install gym! Sometimes difficult on Windows. Google for advise.
import gym
import numpy as np
import random
import math

In [3]:
def BellmanStep(Ra, V, gamma, P):
    Vn = [x[:] for x in V] # todo fix 
    
    for row in range(len(V)):
        
        for col in range(len(V[0])):
            maxL = []  
            s = (row,col)
            E = (s[0]+1,s[1])
            W = (s[0]-1,s[1])
            N = (s[0],s[1]+1)
            S = (s[0],s[1]-1)
            action = [E,W,N,S]
            
            for sp in action:
                maxL.append(value(s,sp, Ra, V, gamma, P))
            maxValue = max(maxL)
            Vn[row][col] = maxValue
    
    return Vn                     
    

In [4]:
def Bellman(Ra, V, gamma, P, i):
    
    for x in range(i):
        V = BellmanStep(Ra, V, gamma, P);
        

    return V
    

In [5]:
def Bellman(Ra, V, gamma, P):
    while(True):
        Vn = BellmanStep(Ra, V, gamma, P);
        if (Vn == V):
            break;
        V = Vn
        
    return V

In [6]:
Ra = [[0,0,0],[0,10,0],[0,0,0]]
V = [[0,0,0],[0,0,0],[0,0,0]] 
gamma = 0.9
P = 0.8
i = 10000

Bellman(Ra, V, gamma, P)

[[45.61292366170413, 51.948051948051926, 45.61292366170413],
 [51.948051948051926, 48.05194805194803, 51.948051948051926],
 [45.61292366170413, 51.948051948051926, 45.61292366170413]]

In [7]:
def bellQ(s, sp, Ra, V, gamma):
    if(sp == -1):
        sp = s
    if(sp == len(V)):
        sp = s
    
    return (Ra[s][sp]+ gamma * V[sp])

In [8]:
def BellmanQstep(Ra, V, gamma):
    Vn = V[:] # todo fix 
    actionTable = []
    for s in range(len(V)):
        maxL = []
        step = s +1
        back = 0
        action = [step,back]
        for sp in action:
            maxL.append(bellQ(s, sp, Ra, V, gamma))
        maxValue = max(maxL)
        Vn[s] = maxValue
        actionTable.append(maxL)
                   
    return (Vn,actionTable)    

In [9]:
def BellmanQ(Ra, V, gamma):
    Vn = ()
    while(True):
        Vn = BellmanQstep(Ra, V, gamma);
        if (Vn[0] == V):
            break;
        V = Vn[0]
        
    return Vn

In [10]:
Ra = [[2,0,0,0,0],[2,0,0,0,0],[2,0,0,0,0],[2,0,0,0,0],[2,0,0,0,10]]
V = [0,0,0,0,0]
gamma = 0.95
BellmanQ(Ra, V, gamma)[1]

[[162.9012499999996, 156.7561874999996],
 [171.4749999999996, 156.7561874999996],
 [180.49999999999957, 156.7561874999996],
 [189.99999999999955, 156.7561874999996],
 [199.99999999999955, 156.7561874999996]]

In [11]:
import gym

In [12]:
env = gym.make('NChain-v0',slip=0.0)
env.reset()

0

In [13]:
num_episodes = 1000 #20000 #60000
gamma = 0.95 #0.99
learning_rate = 0.1 #0.95 #0.85
epsilon = 0.5#1 #0.15 #0.1

# initialize the Q table
Q = np.zeros([5, 2])
Q

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [14]:
Q

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [15]:
for _ in range(num_episodes):
	state = env.reset()
	done = False
	while done == False:
        # First we select an action:
		if random.uniform(0, 1) < epsilon: # Flip a skewed coin
			action = env.action_space.sample() # Explore action space
		else:
			action = np.argmax(Q[state,:]) # Exploit learned values
        # Then we perform the action and receive the feedback from the environment
		new_state, reward, done, info = env.step(action)
        # Finally we learn from the experience by updating the Q-value of the selected action
		update = reward + (gamma*np.max(Q[new_state,:])) - Q[state, action]
		Q[state,action] += learning_rate*update 
		state = new_state

In [16]:
Q

array([[162.90125  , 156.7561875],
       [171.475    , 156.7561875],
       [180.5      , 156.7561875],
       [190.       , 156.7561875],
       [200.       , 156.7561875]])